In [37]:
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt

In [62]:
dataDir = './data/'
glassData = 'glass.txt'
glassNames = ['id','RI','Na', 'Mg', 'Al', 'Si', 'K', 'Ca', 'Ba', 'Fe', 'type']
vote84Data = 'house-votes-84.data'
vote84Names = ['class', 'infant', 'water', 'budget', 'doctorfee','salvador',
              'religion', 'satellite', 'contras', 'missile', 'immigration',
              'synfuels', 'education', 'superfund', 'crime', 'exports',
              'ZAF']

In [64]:
################################################################################
raw = pd.read_csv( os.path.join(dataDir, vote84Data), names=vote84Names )


In [96]:
z = raw.groupby('class').agg({i:'value_counts' for i in raw.columns[1:]})
z

infant  water  budget  doctorfee  salvador  religion  satellite  \
democrat   ?       9     28       7          8        12         9          8   
           n     102    119      29        245       200       135         59   
           y     156    120     231         14        55       123        200   
republican ?       3     20       4          3         3         2          6   
           n     134     73     142          2         8        17        123   
           y      31     75      22        163       157       149         39   

              contras  missile  immigration  synfuels  education  superfund  \
democrat   ?        4       19            4        12         18         15   
           n       45       60          139       126        213        179   
           y      218      188          124       129         36         73   
republican ?       11        3            3         9         13         10   
           n      133      146           73       138         20         22   
           y       24       19           92        21        135        136   

              crime  exports  ZAF  
democrat   ?     10       16   82  
           n    167       91   12  
           y     90      160  173  
republican ?      7       12   22  
           n      3      142   50  
           y    158       14   96

In [94]:
raw.groupby('class').agg(pd.Series.mode)

,infant,water,budget,doctorfee,salvador,religion,satellite,contras,missile,immigration,synfuels,education,superfund,crime,exports,ZAF
class,,,,,,,,,,,,,,,,
democrat,y,y,y,n,n,n,y,y,y,n,y,n,n,n,y,y
republican,n,y,n,y,y,y,n,n,n,y,n,y,y,y,n,y


In [53]:
################################################################################
raw = pd.read_csv( os.path.join(dataDir, glassData), names=glassNames )

# adding some binary classes
raw['window'] = raw['type'] < 5 # type 1-4 are windows
raw['buildWin'] = raw['type'] < 3 # type 1 & 2 are building windows
raw['floatProc'] = raw['type'].isin([1,3]) # 1 & 3 are float processed glass
feature = ['RI','Na', 'Mg', 'Al', 'Si', 'K', 'Ca', 'Ba', 'Fe']

In [59]:
raw.groupby(['window'])[feature].mean()

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe
window,,,,,,,,,
False,1.517638,14.066667,0.733725,1.966667,72.855490,0.559608,9.060196,0.639216,0.023137
True,1.518593,13.201718,3.294908,1.281656,72.586933,0.477485,8.924663,0.029816,0.067607


In [61]:
raw.groupby(['window'])[feature].median()

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe
window,,,,,,,,,
False,1.51711,14.23,0.00,1.94,73.05,0.04,8.93,0.53,0.0
True,1.51775,13.21,3.54,1.29,72.74,0.57,8.55,0.00,0.0


In [30]:
raw

,id,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,type,window
0,1,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.00,0.00,1,True
1,2,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.00,0.00,1,True
2,3,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.00,0.00,1,True
3,4,1.51766,13.21,3.69,1.29,72.61,0.57,8.22,0.00,0.00,1,True
4,5,1.51742,13.27,3.62,1.24,73.08,0.55,8.07,0.00,0.00,1,True
5,6,1.51596,12.79,3.61,1.62,72.97,0.64,8.07,0.00,0.26,1,True
6,7,1.51743,13.30,3.60,1.14,73.09,0.58,8.17,0.00,0.00,1,True
7,8,1.51756,13.15,3.61,1.05,73.24,0.57,8.24,0.00,0.00,1,True
8,9,1.51918,14.04,3.58,1.37,72.08,0.56,8.30,0.00,0.00,1,True
9,10,1.51755,13.00,3.60,1.36,72.99,0.57,8.40,0.00,0.11,1,True
